In [3]:
import meep as mp 
import numpy as np
from meep.materials import Au
from matplotlib import pyplot as plt

def rec_grating_sim(p, t, w, resol):

    resolution = resol  # pixels/μm

    dpml = 1.0
    dsub = 2.0
    dpad = 2.0
    gp = p
    gw = w
    gt = t

    wvl_min = 0.45
    wvl_max = 0.85
    fmin = 1 / wvl_max
    fmax = 1 / wvl_min
    fcen = 0.5 * (fmax + fmin)
    df = fmax - fmin
    nfreq = 200

    glass = mp.Medium(index=2)

    k_point = mp.Vector3() #bloch periodic boundary

    pml_layers = [mp.PML(thickness=dpml, direction=mp.X)]

    sx = dsub + dpad + 2*dpml
    sy = gp #+ gw
    
    cell_size = mp.Vector3(sx,sy)

    src_pt = mp.Vector3(x=-1)
    sources = [mp.Source(mp.GaussianSource(fcen, fwidth=df),
                         component=mp.Ey,
                         center=src_pt,
                         size=mp.Vector3(y=sy))]
    
    geometry = [mp.Block(material=glass,
                         size=mp.Vector3(dpml+dsub,sy,mp.inf),
                         center=mp.Vector3(-0.5*sx+0.5*(dpml+dsub)))]
    
    
    #symmetries=[mp.Mirror(mp.Y)]


    sim = mp.Simulation(resolution=resolution,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        geometry=geometry,
                        k_point=k_point,
                        sources=sources,
                        #symmetries=symmetries,
                        ensure_periodicity=True,dimensions=2)
    #sim.plot2D()

    incident_pt = mp.Vector3(x=1)
    incident_region = mp.FluxRegion(center=incident_pt, size=mp.Vector3(y=sy))
    incident_flux_monitor = sim.add_flux(fcen,df,nfreq,incident_region)

    sim.run(until_after_sources=100)

    
    incident_flux_data = sim.get_flux_data(incident_flux_monitor)
    incident_flux = mp.get_fluxes(incident_flux_monitor)
    
    
    sim.reset_meep()
    
    incident_region = mp.FluxRegion(center=incident_pt, size=mp.Vector3(y=sy))

    geometry.append(mp.Block(material=Au,
                             size=mp.Vector3(gt,0.5*(gp-gw), mp.inf),
                             center=mp.Vector3(-0.5*sx+0.5*gt+dpml+dsub, -0.25*gp)))
    geometry.append(mp.Block(material=Au,
                             size=mp.Vector3(gt,0.5*(gp-gw), mp.inf),
                             center=mp.Vector3(-0.5*sx+0.5*gt+dpml+dsub, 0.25*gp)))
    
    
    sim = mp.Simulation(resolution=resolution,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        geometry=geometry,
                        k_point=k_point,
                        sources=sources,
                        #symmetries=symmetries,
                        ensure_periodicity=True,dimensions=2)
    
    transmission_pt = mp.Vector3(x=1)
    transmission_region = mp.FluxRegion(center=transmission_pt, size=mp.Vector3(y=sy))
    transmission_flux_monitor = sim.add_flux(fcen,df,nfreq,transmission_region)

    #sim.plot2D()

    
    sim.run(until_after_sources=100)
    
    transmitted_flux = mp.get_fluxes(transmission_flux_monitor)
    T = abs(np.divide(transmitted_flux, incident_flux))
    
    freqs = mp.get_flux_freqs(transmission_flux_monitor)
    wvls = np.divide(1000, freqs)
    
    per = gp*1000
    plt.plot(wvls, T, label='p = %d nm' % per)

    
    # Adicione um título ao gráfico
    plt.title('')
    
    # Adicione rótulos aos eixos
    plt.ylabel('Transmission (a.u.)')
    plt.xlabel('Wavelength (nm)')
    
    # Adicione uma legenda
    plt.legend()
    
    plt.savefig('p=%d'%per)

    plt.close()
    
    with open("p=%dnm.txt"%per, "w") as file:
        # Passo 3: Iterar sobre os elementos dos arrays e escrever no formato desejado
        for x, y in zip(wvls, T/T.max()):
            file.write(f"{x} {y}\n")  # Escreve no formato "x y" e adiciona uma nova linha

    return

#rec_grating_sim(0.3, 0.17, 0.06, 50)
#rec_grating_sim(0.4, 0.17, 0.06, 50)
rec_grating_sim(0.5, 0.17, 0.06, 50)
#rec_grating_sim(0.7, 0.17, 0.06, 50)
#rec_grating_sim(0.9, 0.17, 0.06, 50)


-----------
Initializing structure...
time for choose_chunkdivision = 0.000247002 s
Working in 2D dimensions.
Computational cell is 6 x 0.5 x 0 with resolution 50
     block, center = (-1.5,0,0)
          size (3,0.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
time for set_epsilon = 0.00974917 s
-----------


FloatProgress(value=0.0, description='0% done ', max=109.5625)

run 0 finished at t = 109.57000000000001 (10957 timesteps)
-----------
Initializing structure...
time for choose_chunkdivision = 4.29153e-05 s
Working in 2D dimensions.
Computational cell is 6 x 0.5 x 0 with resolution 50
     block, center = (-1.5,0,0)
          size (3,0.5,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
     block, center = (0.085,-0.125,0)
          size (0.17,0.22,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (0.085,0.125,0)
          size (0.17,0.22,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.0157158 s
lorentzian susceptibility: frequency=10.7433, gamma=1.78571
lorentzian susceptibility: frequency=3.47141, gamma=2.01155
lorentzian susceptibility: frequency=2.39466, gamma=0.701702
lorentzian susceptibility: frequency=0.66944, gamma=0.278261
loren

FloatProgress(value=0.0, description='0% done ', max=109.5625)

run 0 finished at t = 109.57000000000001 (10957 timesteps)
